In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os, sys
pylibs =  ':/content/drive/My Drive/Colab_Notebooks/Severstal_CV_Task/python-libs'
os.environ['PYTHONPATH'] += pylibs
sys.path.append(pylibs[1:])
os.chdir("drive/My Drive/Colab_Notebooks/Severstal_CV_Task/training")

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import Model

from segmodel.data_generation import BatchGenerator
from segmodel.utils import DATASET_DIR, ARR_FILE_FORMAT, find_and_load_model
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from pylibs import img_utils

In [3]:
width, height = 256, 256
batch_size = 8
model_num = 1
n_classes = 4
val_img_arr_name = 'dataset1_test'

img_arr_path = DATASET_DIR / (ARR_FILE_FORMAT % val_img_arr_name)
mask_arr_path = DATASET_DIR / (ARR_FILE_FORMAT % (val_img_arr_name + '_masks'))
model: Model = find_and_load_model(model_num, n_classes)

In [4]:
val_gen = BatchGenerator(img_arr_path, mask_arr_path, batch_size, shuffle=False)
predictions = model.predict(val_gen)
labels = val_gen.masks

In [5]:
val_gen = BatchGenerator(img_arr_path, mask_arr_path, batch_size, shuffle=False, apply_normalization=False)

def show_img(img, mask):
  figsize = (15, 10)
  alpha = .9
  img_show = img_utils.apply_mask(img, mask[..., 1], (0, 255, 0), alpha)
  img_utils.show_img(img_show, figsize)
  img_show = img_utils.apply_mask(img, mask[..., 2], (0, 0, 255), alpha)
  img_utils.show_img(img_show, figsize)
  img_show = img_utils.apply_mask(img, mask[..., 3], (0, 128, 255), alpha)
  img_utils.show_img(img_show, figsize)

i = 0
limit = 10
for batch_x, batch_y in val_gen:
    for img, mask in zip(batch_x, batch_y):
        print('label')
        show_img(img, mask)
        pred_mask = predictions[i] > 0.5
        print('pred')
        show_img(img, pred_mask)
        i += 1
        if i > limit:
          break

Output hidden; open in https://colab.research.google.com to view.

In [16]:
shape = predictions.shape
thr = 0.5
pred_squeeze = (predictions.reshape(shape[0] * width * height, n_classes) > thr).argmax(axis=1)
labels_squeeze = labels.reshape(shape[0] * width * height, n_classes).argmax(axis=1)
pred_squeeze.shape, labels_squeeze.shape

((3538944,), (3538944,))

In [20]:
def pretty_print_num(*x):
  end = ', '
  for i, (name, a) in enumerate(x):
    a = (a * 100).round(1)
    if i == len(x) - 1:
      end = '\n'
    print(f"{name}: {a}", end=end)

def print_cm(normalize):
  cm = confusion_matrix(labels_squeeze, pred_squeeze, normalize=normalize)
  pretty_print_num(("confusion matrix", cm))

def print_metrics(average):
  labels = range(1, n_classes)
  f1 = f1_score(labels_squeeze, pred_squeeze, average=average, labels=labels)
  prec = precision_score(labels_squeeze, pred_squeeze, average=average, labels=labels)
  rec = recall_score(labels_squeeze, pred_squeeze, average=average, labels=labels)
  pretty_print_num(("f1 score", f1), ("precision", prec), ("recall", rec))

In [21]:
print_cm('pred')
print_cm('true')
print_cm('all')
print_metrics(None)
print_metrics('micro')
print_metrics('macro')
print_metrics('weighted')

confusion matrix: [[99.1 38.6 27.8 76.9]
 [ 0.3 60.9  0.2  0.2]
 [ 0.3  0.4 72.   0. ]
 [ 0.2  0.   0.  22.9]]
confusion matrix: [[99.4  0.   0.2  0.4]
 [92.2  7.   0.4  0.3]
 [35.6  0.  64.4  0. ]
 [68.4  0.   0.  31.6]]
confusion matrix: [[97.7  0.   0.2  0.4]
 [ 0.3  0.   0.   0. ]
 [ 0.3  0.   0.6  0. ]
 [ 0.2  0.   0.   0.1]]
f1 score: [12.6 68.  26.5], precision: [60.9 72.  22.9], recall: [ 7.  64.4 31.6]
f1 score: 49.1, precision: 54.1, recall: 45.0
f1 score: 35.7, precision: 51.9, recall: 34.3
f1 score: 47.2, precision: 59.0, recall: 45.0
